## Pasos a seguri para la realizacion del proyecto:
1. Instalar y cargar las librerías necesarias.

2. Cargar y explorar los datos (train.csv y test.csv).

3️. Preprocesamiento y limpieza de datos.

4️. Dividir en conjunto de entrenamiento y validación.

5️. Entrenar modelos de regresión y evaluar RMSE.

6️. Optimizar el modelo para mejorar la precisión.

7️. Hacer predicciones finales y generar submission.csv.


### Instalar y Cargar Librerías

In [34]:
pip install pandas numpy scikit-learn matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [36]:
# Importamos librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error


### Cargar y Explorar los Datos


In [37]:
# Datasets
train_df = pd.read_csv("/Users/josevillarroel/Desktop/Curso/DS102024/3-MachineLearning/2-Supervisado/Ejercicios/train copy.csv")
test_df = pd.read_csv("/Users/josevillarroel/Desktop/Curso/DS102024/3-MachineLearning/2-Supervisado/Ejercicios/data/test copy.csv")

# Información general
print("🔍 Información del dataset de entrenamiento:")
print(train_df.info())

print("\n🔍 Información del dataset de prueba:")
print(test_df.info())

# Primeras filas
print("\n📊 Primeras filas del train.csv:")
print(train_df.head())

print("\n📊 Primeras filas del test.csv:")
print(test_df.head())


🔍 Información del dataset de entrenamiento:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB
None

🔍 Información del dataset de prueba:
<class 'pandas.core.frame.DataFrame'>
R

### Que no se nos olvide que queremos predecir la variable Price_euros 

## Preprocesamiento y Limpieza de Datos

#### Conversión de datos:
Algunas columnas están en formato de texto pero deberían ser numéricas:

In [38]:
# Convertir 'Ram' y 'Weight' a valores numéricos
train_df["Ram"] = train_df["Ram"].str.replace("GB", "").astype(int)
test_df["Ram"] = test_df["Ram"].str.replace("GB", "").astype(int)

train_df["Weight"] = train_df["Weight"].str.replace("kg", "").astype(float)
test_df["Weight"] = test_df["Weight"].str.replace("kg", "").astype(float)


### Vamos a extraer características de CPU y Memoria

In [39]:
# Extraer la marca del procesador
train_df["Cpu_brand"] = train_df["Cpu"].apply(lambda x: x.split()[0])
test_df["Cpu_brand"] = test_df["Cpu"].apply(lambda x: x.split()[0])

# Extraer tipo de almacenamiento de 'Memory'
def process_memory(memory):
    if "SSD" in memory:
        return "SSD"
    elif "HDD" in memory:
        return "HDD"
    elif "Flash" in memory:
        return "Flash"
    else:
        return "Other"

train_df["Memory_type"] = train_df["Memory"].apply(process_memory)
test_df["Memory_type"] = test_df["Memory"].apply(process_memory)


### Eliminar columnas irrelevantes

In [40]:
drop_columns = ["laptop_ID", "Product", "ScreenResolution", "Cpu", "Memory", "Gpu"]
train_df.drop(columns=drop_columns, inplace=True)
test_df.drop(columns=drop_columns, inplace=True)


In [41]:
train_df

,Company,TypeName,Inches,Ram,OpSys,Weight,Price_euros,Cpu_brand,Memory_type
0,Dell,Notebook,15.6,8,Windows 10,2.36,889.00,Intel,SSD
1,Lenovo,Notebook,15.6,8,No OS,2.20,519.00,Intel,HDD
2,Dell,2 in 1 Convertible,13.3,8,Windows 10,1.24,1813.00,Intel,SSD
3,Dell,2 in 1 Convertible,15.6,8,Windows 10,1.56,1049.00,Intel,SSD
4,LG,Ultrabook,14.0,8,Windows 10,0.98,1899.00,Intel,SSD
...,...,...,...,...,...,...,...,...,...
907,Lenovo,Notebook,15.6,4,Windows 10,2.20,399.00,AMD,SSD
908,Lenovo,2 in 1 Convertible,13.3,16,Windows 10,1.30,1799.00,Intel,SSD
909,Dell,Notebook,15.6,16,Windows 10,1.93,1537.39,Intel,SSD
910,Lenovo,Gaming,15.6,8,No OS,2.40,989.00,Intel,SSD


In [42]:
test_df

,Company,TypeName,Inches,Ram,OpSys,Weight,Cpu_brand,Memory_type
0,Asus,Notebook,15.6,8,Windows 10,2.00,Intel,SSD
1,Asus,Notebook,14.0,8,Windows 10,2.00,Intel,SSD
2,Mediacom,Notebook,13.3,4,Windows 10,1.35,Intel,Flash
3,Apple,Ultrabook,15.4,16,macOS,1.83,Intel,SSD
4,HP,Ultrabook,15.6,8,Windows 10,1.84,Intel,SSD
...,...,...,...,...,...,...,...,...
386,Lenovo,Notebook,13.3,8,Windows 10,1.44,Intel,SSD
387,Dell,Ultrabook,13.3,8,Windows 10,1.20,Intel,SSD
388,Lenovo,Notebook,15.6,6,Windows 10,2.40,Intel,SSD
389,HP,Notebook,14.0,8,Windows 7,1.43,Intel,SSD


### Convertir variables categóricas en dummies

In [43]:
train_df = pd.get_dummies(train_df, drop_first=True)
test_df = pd.get_dummies(test_df, drop_first=True)

# Nos aseguramos que ambas tablas tengan las mismas columnas
missing_cols = set(train_df.columns) - set(test_df.columns)
for col in missing_cols:
    test_df[col] = 0  # Agregar columnas faltantes con valor 0

# Ordenamos columnas para que coincidan con el modelo
test_df = test_df[train_df.drop(columns=["Price_euros"]).columns]


## División en Train y Validación

In [44]:
# Separamos variables predictoras (X) y objetivo (y)
X = train_df.drop(columns=["Price_euros"])
y = train_df["Price_euros"]

# División en conjuntos de entrenamiento (80%) y validación (20%)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


## Entrenar Modelos y Evaluar RMSE

In [45]:
# Función para evaluar modelos usando RMSE
def evaluate_model(model, X_train, y_train, X_valid, y_valid):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_pred)  # Calcular MSE
    rmse = np.sqrt(mse)  # Calcular RMSE manualmente
    return rmse


In [58]:
# Calculamos RMSE del modelo Random Forest
rmse_random_forest = evaluate_model(best_model, X_train, y_train, X_valid, y_valid)

# Mostrar el RMSE calculado
rmse_random_forest

np.float64(372.3285628657684)

#### El RMSE del modelo Random Forest es: 372.33

In [59]:
# Guardamos el archivo en una ruta accesible
submission_path = "/Users/josevillarroel/Desktop/Curso/DS102024/3-MachineLearning/2-Supervisado/Ejercicios/data/sample_submission copy.csv"
submission.to_csv(submission_path, index=False)

print(f"✅ Archivo generado con éxito: {submission_path}")


✅ Archivo generado con éxito: /Users/josevillarroel/Desktop/Curso/DS102024/3-MachineLearning/2-Supervisado/Ejercicios/data/sample_submission copy.csv


### A este punto, seleccionamos el modelo con menor RMSE.



## Generar Predicciones Finales

In [60]:
# Entrenamos el mejor modelo (Ejemplo: Random Forest)
best_model = RandomForestRegressor(n_estimators=100, random_state=42)
best_model.fit(X, y)

# Hacemos predicciones en test.csv
test_predictions = best_model.predict(test_df)

# Creamos el archivo de submission.csv
submission = pd.DataFrame({"laptop_ID": test_df.index, "Price_euros": test_predictions})
submission.to_csv("/Users/josevillarroel/Desktop/Curso/DS102024/3-MachineLearning/2-Supervisado/Ejercicios/data/sample_submission copy.csv", index=False)

print("✅ Archivo submission.csv generado con éxito.")


✅ Archivo submission.csv generado con éxito.
